In [1]:
# LazyJSON would be faster, but having issues installing to 0.6.1
using Query, DataFrames, CSV, JSON

Import list of stations, departures, arrivals

In [2]:
haltestellen = CSV.read("data/haltestellen-in-metro.csv")
head(haltestellen)

,xtf_id,Nummer,Name,Abkuerzung,DatenherrA,TUNummer,TUAbkuerzu,Betriebspu,Verkehrsmi,Hoehe,GdeNummer,GdeName,BeginnGuel,EndeGuelti,Bearbeitun,Stand,rUebergeor
1,ch14uvag00047298,8503391,"Goldingen, Vordersagen",missing,missing,7,PAG,Haltestelle,Bus,780,3342,Eschenbach (SG),2014/12/01,missing,2017/12/01,2017/12/10,missing
2,ch14uvag00058578,8503479,"Schaffhausen, Kreuz",missing,missing,165,RVSH,Haltestelle,Bus,385,2939,Schaffhausen,2014/12/01,missing,2017/12/01,2017/12/10,missing
3,ch14uvag00042993,8591837,"Cham, Lorzenpark",missing,missing,185,ZVB,Haltestelle,Bus,432,1702,Cham,2014/12/01,missing,2017/12/01,2017/12/10,missing
4,ch14uvag00061208,8573492,"Thayngen, Bahnhof",missing,missing,165,RVSH,Haltestelle,Bus,417,2920,Thayngen,2014/12/01,missing,2017/12/01,2017/12/10,missing
5,ch14uvag00060992,8582713,"Tägerig, Weidhof",missing,missing,7,PAG,Haltestelle,Bus,378,4077,Tägerig,2014/12/01,missing,2017/12/01,2017/12/10,missing
6,ch14uvag00063496,8592431,"Weggis, Post",missing,missing,340,AAGS,Haltestelle,Bus,445,1069,Weggis,2014/12/01,missing,2017/12/01,2017/12/10,missing


In [3]:
open("data/Metroregion.geojson", "r") do f
    global metroregion
    metroregion = JSON.parse(readstring(f))
end

Dict{String,Any} with 4 entries:
  "name"     => "Metroregion"
  "features" => Any[Dict{String,Any}(Pair{String,Any}("properties", Dict{String…
  "type"     => "FeatureCollection"
  "crs"      => Dict{String,Any}(Pair{String,Any}("properties", Dict{String,Any…

In [4]:
metronumbers = [ i["properties"]["BFS_NUMMER"] for i in metroregion["features"] ]
length(metronumbers)

343

Run a query to filter to the MKZ communes.

In [5]:
ZURICH_HB = "8503000"
ZURICH_SZU = "8503088"
q1 = @from i in haltestellen begin
    @where i.GdeNummer in metronumbers && i.Nummer != ZURICH_HB && i.Nummer != ZURICH_SZU
    @select {id=i.Nummer, name=i.Name, geoid=i.GdeNummer, geoname=i.GdeName}
    @collect DataFrame
end

,id,name,geoid,geoname
1,8503391,"Goldingen, Vordersagen",3342,Eschenbach (SG)
2,8503479,"Schaffhausen, Kreuz",2939,Schaffhausen
3,8591837,"Cham, Lorzenpark",1702,Cham
4,8573492,"Thayngen, Bahnhof",2920,Thayngen
5,8582713,"Tägerig, Weidhof",4077,Tägerig
6,8592431,"Weggis, Post",1069,Weggis
7,8580276,"Wohlen AG, Luegisland",4082,Wohlen (AG)
8,8576957,"Tänikon, Forschungsanstalt",4551,Aadorf
9,8590957,"Winterthur, Hedy Hahnloser",230,Winterthur
10,8506004,Kollbrunn,231,Zell (ZH)


Load the current travel plan in [GTFS format](https://opentransportdata.swiss/de/cookbook/gtfs/#Wann_gibt_es_neue_GTFS_Static_Files)

In [6]:
stoptimes = CSV.read("data/gtfsfp/stop_times.txt")
head(stoptimes)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type
1,1.TA.1-1-A-j18-1.1.H,01:00:00,01:00:00,8503000:0:41/42,1,0,0
2,1.TA.1-1-A-j18-1.1.H,01:02:00,01:02:00,8503020:0:4,2,0,0
3,1.TA.1-1-A-j18-1.1.H,01:05:00,01:05:00,8503001:0:3,3,0,0
4,1.TA.1-1-A-j18-1.1.H,01:07:00,01:07:00,8503509:0:3,4,0,0
5,1.TA.1-1-A-j18-1.1.H,01:10:00,01:10:00,8503512:0:1,5,0,0
6,1.TA.1-1-A-j18-1.1.H,01:13:00,01:15:00,8503508:0:3,6,0,0


Clean up the stop identifiers and parse timestamps.

In [9]:
function get_23_time(the_time)
    bad_hour = parse(Int64, the_time[1:2])
    if bad_hour > 23
        the_time = replace(the_time, r"..(:..:..)", s"\1")
        the_time = string(bad_hour-24) * the_time
    end
    DateTime(the_time, dateformat"HH:MM:SS")
end

stoptimes[:stop_id] = [ split(i[:stop_id],":")[1] for i in eachrow(stoptimes) ]
head(stoptimes)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type
1,1.TA.1-1-A-j18-1.1.H,01:00:00,01:00:00,8503000,1,0,0
2,1.TA.1-1-A-j18-1.1.H,01:02:00,01:02:00,8503020,2,0,0
3,1.TA.1-1-A-j18-1.1.H,01:05:00,01:05:00,8503001,3,0,0
4,1.TA.1-1-A-j18-1.1.H,01:07:00,01:07:00,8503509,4,0,0
5,1.TA.1-1-A-j18-1.1.H,01:10:00,01:10:00,8503512,5,0,0
6,1.TA.1-1-A-j18-1.1.H,01:13:00,01:15:00,8503508,6,0,0


Calculate time difference

In [38]:
cur_trip_id = ""
cur_trip_start = 0
diff_times = Dict()

for i in eachrow(stoptimes)
    if cur_trip_id != i[:trip_id]
        cur_trip_id = ""
    end
    cur_stop_id = i[:stop_id]
    if cur_stop_id == ZURICH_HB
        cur_trip_id = i[:trip_id]
        cur_trip_start = get_23_time(i[:departure_time])
    elseif cur_trip_id != ""
        cur_trip_end = get_23_time(i[:arrival_time])
        if cur_trip_start != cur_trip_end && cur_trip_end != 0 && cur_trip_start != 0
            trip_time_diff = abs(Dates.value(cur_trip_end) - Dates.value(cur_trip_start))
            #if haskey(diff_times, cur_stop_id)
            #    diff_times[cur_stop_id] = Integer((diff_times[cur_stop_id] + trip_time_diff) / 2)
            #else
                diff_times[cur_stop_id] = Integer(trip_time_diff)
            #end
        end
    end
end

Dates.epochms2datetime(diff_times["8500309"])

In [53]:
q1[:min_to_hb] = [ 
    if haskey(diff_times, string(stop[:id]))
        diff_times[string(stop[:id])]
    else
        0
    end
    for stop in eachrow(q1)
]
head(q1)

,id,name,geoid,geoname,min_to_hb
1,8503391,"Goldingen, Vordersagen",3342,Eschenbach (SG),0
2,8503479,"Schaffhausen, Kreuz",2939,Schaffhausen,0
3,8591837,"Cham, Lorzenpark",1702,Cham,0
4,8573492,"Thayngen, Bahnhof",2920,Thayngen,0
5,8582713,"Tägerig, Weidhof",4077,Tägerig,0
6,8592431,"Weggis, Post",1069,Weggis,0


In [65]:
q2 = @from i in q1 begin
    @where i.min_to_hb > 0
    @select {i.id,i.name,i.geoid,i.geoname,i.min_to_hb}
    @collect DataFrame
end
q2

,id,name,geoid,geoname,min_to_hb
1,8503400,Bülach,53,Bülach,1020000
2,8506013,Aadorf,4551,Aadorf,2640000
3,8503511,Neuenhof,4034,Neuenhof,1320000
4,8503130,Hinwil,117,Hinwil,2400000
5,8503141,Küsnacht Goldbach,154,Küsnacht (ZH),720000
6,8503101,Küsnacht ZH,154,Küsnacht (ZH),900000
7,8503204,Horgen,133,Horgen,1260000
8,8503205,Au ZH,142,Wädenswil,1440000
9,8502306,Hünenberg Chämleten,1703,Hünenberg,2460000
10,8503020,Zürich Hardbrücke,261,Zürich,120000


Statistical summaries (max, min, average)

In [69]:
describe(q2[:min_to_hb])

Summary Stats:
Mean:           5446800.000000
Minimum:        120000.000000
1st Quartile:   1020000.000000
Median:         1620000.000000
3rd Quartile:   2160000.000000
Maximum:        84780000.000000
Length:         150
Type:           Int64


Visualisations

Export